In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
# ------ 1. Preparing data ------
df = pd.read_csv('data/raw/icd.csv', delimiter=';')

# 1.1 filter
df = df[['time', '2_variable_label', '2_variable_attribute_code', '2_variable_attribute_label','2_variable_code', 'value']]

df.rename(columns={
    'time': 'year',
    '2_variable_label': 'icd_form_type',
    '2_variable_code': 'icd_form',
    '2_variable_attribute_code': 'icd_code',
    '2_variable_attribute_label': 'diagnosis_category',
    'value': 'count',
}, inplace=True)



# 1.2 type conversion of column 'value' since it is a object type
mapping = str.maketrans({',': '.', '-': '0'})
df['count'] = df['count'].astype(str).str.translate(mapping)
df['count'] = pd.to_numeric(df['count'], errors='raise')

# checks
df.head()

,year,icd_form_type,icd_code,diagnosis_category,icd_form,count
0,2021,ICD-10 (1-Steller) Nebendiagnosen,ICD10-D00-D48,Übrige Neubildungen,ICD10D,594620
1,2019,ICD-10 (1-Steller) Nebendiagnosen,ICD10-J00-J99,Krankheiten des Atmungssystems,ICD10D,4914020
2,2018,ICD-10 (1-Steller) Nebendiagnosen,ICD10-M00-M99,Krankh. d. Muskel-Skelett-Systems u.d.Bindegew...,ICD10D,3969546
3,2017,ICD-10 (1-Steller) Nebendiagnosen,ICD10-C00-C97,Bösartige Neubildungen,ICD10D,1845640
4,2016,ICD-10 (1-Steller) Nebendiagnosen,ICD10-K00-K93,Krankheiten des Verdauungssystems,ICD10D,4856484


In [17]:
df.notna().sum()

year                  20250
icd_form_type         20250
icd_code              20240
diagnosis_category    20250
icd_form              20250
icd_code              20240
count                 20250
dtype: int64

Seems like we found a culprit!
'icd_code' has some missing values
what to do with it?

In [22]:
df[df.isna().any(axis=1)]

,year,icd_form_type,icd_code,diagnosis_category,icd_form,count
49,2021,ICD-10 (1-Steller) Nebendiagnosen,NaN,Insgesamt,ICD10D,139788116
93,2019,ICD-10 (1-Steller) Nebendiagnosen,NaN,Insgesamt,ICD10D,120899995
124,2015,ICD-10 (1-Steller) Nebendiagnosen,NaN,Insgesamt,ICD10D,106318408
134,2022,ICD-10 (1-Steller) Nebendiagnosen,NaN,Insgesamt,ICD10D,139414834
173,2018,ICD-10 (1-Steller) Nebendiagnosen,NaN,Insgesamt,ICD10D,118003312
183,2016,ICD-10 (1-Steller) Nebendiagnosen,NaN,Insgesamt,ICD10D,111603570
200,2023,ICD-10 (1-Steller) Nebendiagnosen,NaN,Insgesamt,ICD10D,125171046
205,2024,ICD-10 (1-Steller) Nebendiagnosen,NaN,Insgesamt,ICD10D,121544729
210,2020,ICD-10 (1-Steller) Nebendiagnosen,NaN,Insgesamt,ICD10D,123731421
226,2017,ICD-10 (1-Steller) Nebendiagnosen,NaN,Insgesamt,ICD10D,115354906


In [25]:
# Basic strategy for handling missing values
df['icd_code'].fillna('undefined', inplace=True)
df.notna().sum()

year                  20250
icd_form_type         20250
icd_code              20250
diagnosis_category    20250
icd_form              20250
count                 20250
dtype: int64